<a href="https://colab.research.google.com/github/hadine30/crowd-counting/blob/master/inception_resnet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
! git clone https://github.com/yuyang-huang/keras-inception-resnet-v2.git

Cloning into 'keras-inception-resnet-v2'...
remote: Enumerating objects: 113, done.
remote: Total 113 (delta 0), reused 0 (delta 0), pack-reused 113
Receiving objects: 100% (113/113), 63.09 KiB | 223.00 KiB/s, done.
Resolving deltas: 100% (66/66), done.


In [19]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [0]:
#!pip install tensorflow

In [0]:
!pip install -Uq tensorflow==1.8.0

In [0]:
!pip install -Uq keras==2.2.0

In [0]:
from google.colab import files
#files.download("/content/drive/keras-inception-resnet-v2.zip")

In [24]:
!git clone https://github.com/matterport/Mask_RCNN
import os, sys
os.chdir('/content/Mask_RCNN/samples')
# Root directory of the project
ROOT_DIR = os.path.abspath("../")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.82 MiB | 9.88 MiB/s, done.
Resolving deltas: 100% (568/568), done.


! git clone https://github.com/tensorflow/models
os.chdir('/content/tensorflow/models/blob/master/research/slim/nets')
# Root directory of the project
ROOT_DIR = os.path.abspath("../")

In [0]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

In [26]:
!python3 "/content/keras-inception-resnet-v2/extract_weights.py"

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarning: Passing (type, 1) or 

j'ai modifier 
la ligne 82:MODEL_DIR = './'
la ligne 56 :"reader = tf.train.load_checkpoint(filename)"

In [27]:
from __future__ import print_function

import os
import re
from glob import glob
import numpy as np
import tensorflow as tf
from keras.utils.data_utils import get_file


# regex for renaming the tensors to their corresponding Keras counterpart
re_repeat = re.compile(r'Repeat_[0-9_]*b')
re_block8 = re.compile(r'Block8_[A-Za-z]')


def get_filename(key):
    """Rename tensor name to the corresponding Keras layer weight name.

    # Arguments
        key: tensor name in TF (determined by tf.variable_scope)
    """
    filename = str(key)
    filename = filename.replace('/', '_')
    filename = filename.replace('InceptionResnetV2_', '')

    # remove "Repeat" scope from filename
    filename = re_repeat.sub('B', filename)

    if re_block8.match(filename):
        # the last block8 has different name with the previous 9 occurrences
        filename = filename.replace('Block8', 'Block8_10')
    elif filename.startswith('Logits'):
        # remove duplicate "Logits" scope
        filename = filename.replace('Logits_' , '', 1)

    # from TF to Keras naming
    filename = filename.replace('_weights', '_kernel')
    filename = filename.replace('_biases', '_bias')

    return filename + '.npy'


def extract_tensors_from_checkpoint_file(filename, output_folder='weights'):
    """Extract tensors from a TF checkpoint file.

    # Arguments
        filename: TF checkpoint file
        output_folder: where to save the output numpy array files
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    reader = tf.train.load_checkpoint(filename)

    for key in reader.get_variable_to_shape_map():
        # not saving the following tensors
        if key == 'global_step':
            continue
        if 'AuxLogit' in key:
            continue

        # convert tensor name into the corresponding Keras layer weight name and save
        path = os.path.join(output_folder, get_filename(key))
        arr = reader.get_tensor(key)
        np.save(path, arr)
        print("tensor_name: ", key)


# download TF-slim checkpoint for Inception-ResNet v2 and extract
CKPT_URL = 'http://download.tensorflow.org/models/inception_resnet_v2_2016_08_30.tar.gz'
MODEL_DIR = './'

checkpoint_tar = get_file(
    'inception_resnet_v2_2016_08_30.tar.gz',
    CKPT_URL,
    file_hash='9e0f18e1259acf943e30690460d96123',
    hash_algorithm='md5',
    extract=True,
    cache_subdir='',
    cache_dir=MODEL_DIR)

checkpoint_file = glob(os.path.join(MODEL_DIR, 'inception_resnet_v2_*.ckpt'))[0]
extract_tensors_from_checkpoint_file(checkpoint_file)


tensor_name:  InceptionResnetV2/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/weights
tensor_name:  InceptionResnetV2/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/BatchNorm/moving_variance
tensor_name:  InceptionResnetV2/Repeat_1/block17_13/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
tensor_name:  InceptionResnetV2/Repeat_2/block8_4/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
tensor_name:  InceptionResnetV2/Repeat_2/block8_4/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean
tensor_name:  InceptionResnetV2/Repeat/block35_1/Branch_2/Conv2d_0a_1x1/BatchNorm/moving_variance
tensor_name:  InceptionResnetV2/Repeat_2/block8_4/Branch_0/Conv2d_1x1/weights
tensor_name:  InceptionResnetV2/Repeat_2/block8_4/Branch_0/Conv2d_1x1/BatchNorm/moving_variance
tensor_name:  InceptionResnetV2/Repeat_2/block8_3/Branch_1/Conv2d_0c_3x1/weights
tensor_name:  InceptionResnetV2/Mixed_6a/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean
tensor_name:  InceptionResnetV2/Repeat/block35_9/Branch_1/Conv2d_0a_1x1/BatchNorm/beta

In [28]:
!pip install tf-models-official

In [29]:
!python3 "/content/keras-inception-resnet-v2/load_weights.py"

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarn

inception_resnet_v2.py:::::::::::
:::::::::
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.layers import AveragePooling2D
from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import Concatenate
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import GlobalAveragePooling2D
from tensorflow.python.keras.layers import GlobalMaxPooling2D
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import Lambda
from tensorflow.python.keras.layers import MaxPooling2D
from tensorflow.python.keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras_applications.imagenet_utils import _obtain_input_shape
from keras import backend as K

load_weights.py::::::::
::::::
from tensorflow.keras.models import Model

In [0]:
#!python3 "/content/keras-inception-resnet-v2/test_inception_resnet_v2.py"

In [0]:
!unzip -uq "/content/drive/My Drive/nets.zip" -d "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/slim/"

In [32]:
print(os.listdir("/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/slim/nets"))

['pix2pix.py', 'inception_v1.py', 'alexnet.py', 'inceptionresnetv2.py', 'pix2pix_test.py', 'mobilenet_v1_eval.py', 'inception_v3_test.py', 'lenet.py', '__pycache__', 'inception_resnet_v2.py', 'mobilenet_v1.png', 'mobilenet_v1_test.py', 'nets_factory.py', 'resnet_v1.py', 'mobilenet_v1.py', 'resnet_utils.py', 'overfeat.py', 'vgg.py', 'dcgan.py', 'inception_utils.py', 'inception_v1_test.py', 'dcgan_test.py', 'nasnet', 'post_training_quantization.py', 'cyclegan.py', 'mobilenet', 'resnet_v1_test.py', 'i3d_test.py', 'vgg_test.py', 'alexnet_test.py', 's3dg_test.py', '__init__.py', 'overfeat_test.py', 'i3d.py', 'resnet_v2.py', 'inception_v4_test.py', 'inception.py', 'inception_resnet_v2_test.py', 'inception_v4.py', 'resnet_v2_test.py', 'nets_factory_test.py', 'inception_v2_test.py', 's3dg.py', 'mobilenet_v1.md', 'inception_v3.py', 'inception_v2.py', 'cyclegan_test.py', 'i3d_utils.py', 'cifarnet.py', 'mobilenet_v1_train.py']


In [38]:
from __future__ import print_function

import os

import tensorflow as tf
from tensorflow.contrib import slim
from tensorflow.contrib.slim import nets as nets

import tensorflow as tf; slim = tf.contrib.slim
import numpy as np
from PIL import Image
from keras.models import load_model

from tensorflow.contrib.slim.nets import *

from tensorflow.contrib.slim.nets import inception_resnet_v2 as slim_irv2  # PYHTONPATH should contain the research/slim/ directory in the tensorflow/models repo.
import inception_resnet_v2 as keras_irv2


IMAGES = ['elephant.jpg']
MODEL_DIR = './models'
SLIM_CKPT = os.path.join(MODEL_DIR, 'inception_resnet_v2_2016_08_30.ckpt')
KERAS_CKPT = os.path.join(MODEL_DIR, 'inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
ATOL = 1e-5
VERBOSE = True


def predict_slim(sample_images, print_func=print):
    """
    Code modified from here: [https://github.com/tensorflow/models/issues/429]
    """
    # Setup preprocessing
    input_tensor = tf.placeholder(tf.float32, shape=(None, 299, 299, 3), name='input_image')
    scaled_input_tensor = tf.scalar_mul((1.0 / 255), input_tensor)
    scaled_input_tensor = tf.subtract(scaled_input_tensor, 0.5)
    scaled_input_tensor = tf.multiply(scaled_input_tensor, 2.0)

    # Setup session
    sess = tf.Session()
    arg_scope = slim_irv2.inception_resnet_v2_arg_scope()
    with slim.arg_scope(arg_scope):
        _, end_points = slim_irv2.inception_resnet_v2(scaled_input_tensor, is_training=False)

    # Load the model
    print_func("Loading TF-slim checkpoint...")
    saver = tf.train.Saver()
    saver.restore(sess, SLIM_CKPT)

    # Make prediction
    predict_values = []
    for image in sample_images:
        im = Image.open(image).resize((299, 299))
        arr = np.expand_dims(np.array(im), axis=0)
        y_pred = sess.run([end_points['Predictions']], feed_dict={input_tensor: arr})
        y_pred = y_pred[0].ravel()

        y_pred = y_pred[1:] / y_pred[1:].sum()  # remove background class and renormalize
        print_func("{} class={} prob={}".format(image, np.argmax(y_pred), np.max(y_pred)))
        predict_values.append(y_pred)

    return predict_values


def predict_keras(sample_images, print_func=print):
    # Load the model
    print_func("Loading Keras checkpoint...")
    model = keras_irv2.InceptionResNetV2(weights=None)
    model.load_weights(KERAS_CKPT)

    # Make prediction
    predict_values = []
    for image in sample_images:
        im = Image.open(image).resize((299, 299))
        arr = np.expand_dims(np.array(im), axis=0)
        y_pred = model.predict(keras_irv2.preprocess_input(arr.astype('float32')))
        y_pred = y_pred.ravel()
        print_func("{} class={} prob={}".format(image, np.argmax(y_pred), np.max(y_pred)))
        predict_values.append(y_pred)

    return predict_values


# test whether Keras implementation gives the same result as TF-slim implementation
verboseprint = print if VERBOSE else lambda *a, **k: None
slim_predictions = predict_slim(IMAGES, verboseprint)
keras_predictions = predict_keras(IMAGES, verboseprint)

for filename, y_slim, y_keras in zip(IMAGES, slim_predictions, keras_predictions):
    np.testing.assert_allclose(y_slim, y_keras, atol=ATOL, err_msg=filename)
    verboseprint('{} passed test. (tolerance={})'.format(filename, ATOL))


AttributeError: ignored